In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss
from sklearn.feature_selection import SelectKBest, f_classif

# Open file with pd.read_csv
df_train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
df_test = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")

# Convert 'A' and 'B' values in 'EJ' column to 0 and 1 respectively
df_train['EJ'] = df_train['EJ'].map({'A': 0, 'B': 1})
df_test['EJ'] = df_test['EJ'].map({'A': 0, 'B': 1})

# Split the training data into features (X) and target variable (y)
X_train = df_train.drop(["Class", "Id"], axis=1)  # Exclude non-numeric columns
y_train = df_train["Class"]

# Split the test data into features (X_test)
X_test = df_test.drop("Id", axis=1)

# Identify columns with missing values
columns_with_missing = X_train.columns[X_train.isna().any()].tolist()

# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Perform feature selection using SelectKBest and ANOVA F-value
k = 20  # Number of top features to select
feature_selector = SelectKBest(f_classif, k=k)
X_train_selected = feature_selector.fit_transform(X_train_scaled, y_train)
X_test_selected = feature_selector.transform(X_test_scaled)
selected_feature_indices = feature_selector.get_support(indices=True)
selected_features = X_train.columns[selected_feature_indices]

# Create an SVM model with tuned parameters
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True)

# Train the SVM model
svm.fit(X_train_selected, y_train)

# Predict on the test set
svm_pred = svm.predict(X_test_selected)

# Predict probabilities for each class in the test set
svm_pred_proba = svm.predict_proba(X_test_selected)

# Create a DataFrame to store the predictions
predictions_df = pd.DataFrame({'Id': df_test['Id'],
                               'class_0': svm_pred_proba[:, 0],
                               'class_1': svm_pred_proba[:, 1]})

# Save the predictions to a CSV file
predictions_df.to_csv('submission.csv', index=False)